# Introducción al análisis de series temporales

## ¿Qué es una serie temporal?

 En esencia, una __serie temporal__ es una colección de observaciones realizadas secuencialmente en el tiempo. 
 
- Una serie implica una __colección de observaciones__.
- Temporal implica un tratamiento de forma __secuencial__ (dependencia temporal).
- En general, las observaciones aparecen con un intervalo de tiempo fijo (e.g., cada hora, día, mes). 
- Las series muestreadas irregularmente también se pueden representarse como una serie temporal.

## ¿Por qué se utilizan las series temporales?

Ejemplos de aplicaciones: 

- __Predicción__ de series temporales: predecir los valores futuros de una serie temporal, dados los valores pasados; por ejemplo, predecir la temperatura del día siguiente utilizando los datos de temperatura de los últimos 5 años.


- __Clasificación__ de series temporales: a veces, en lugar de predecir el valor futuro de la serie temporal, también podemos querer predecir una acción en función de valores pasados. Por ejemplo, dado un historial de un electroencefalograma (EEG; seguimiento de la actividad eléctrica en el cerebro) o un electrocardiograma (EKG; seguimiento de la actividad eléctrica en el corazón), necesitamos predecir si el resultado de un EEG o un EKG es normal o anormal.


- __Interpretación__ y __causalidad__: comprender los qué y los porqués de la serie temporal en función de los valores pasados, comprender las interrelaciones entre varias series temporales relacionadas o derivar una inferencia causal en función de los datos de series temporales.

## Ejemplos de series temporales

In [1]:
import pandas as pd
import hvplot.pandas

data = pd.read_csv("examples/pluv_daily_937013.csv", parse_dates=['Date'], index_col=['Date'])
data.hvplot(
    title='Estación pluviométrica (Acumulación diaría)',
    ylabel='(mm)',
    line_width=1,
    ylim=(0,data.values.max())
).opts(active_tools=[])

:Curve   [Date]   (Sum)

In [2]:
data = pd.read_csv(
    "http://geodesy.unr.edu/gps_timeseries/tenv3/IGS14/CONT.tenv3",
    sep="\s+",
    usecols=['yyyy.yyyy', '__east(m)']).set_index('yyyy.yyyy')

data['__east(m)'] = data['__east(m)'] * 1000

data['__east(m)'].hvplot(
    title="Estación GPS CONT (Constitución, Chile)",
    ylabel='Este (mm)',
    xlabel='Date',
    line_width=1,
    grid=True
).opts(active_tools=[])

:Curve   [yyyy.yyyy]   (__east(m))

In [3]:
data = pd.read_csv('examples/WELLS_BABJ.txt', sep="\s+")
data = data[data['Date'] < 2021].set_index('Date')

data.hvplot(
    title='São Francisco River Basin, Brazil',
    ylabel='Height(m)',
    line_width=1,
    grid=True
).opts(active_tools=[])

:Curve   [Date]   (Height)

## Componentes de una serie temporal

A menudo se supone que una serie temporal está compuesta por tres componentes:
- __Tendencia__: la dirección a largo plazo.
- __Estacionalidad__: el comportamiento periódico.
- __Residuos__: las fluctuaciones irregulares.

### Tendencia (_trend_)

- La tendencia __captura la dirección general__ de la serie temporal.
- Por ejemplo, el (aumento o disminución) número de pasajeros a lo largo de los años a pesar de las fluctuaciones estacionales.
- La tendencia puede ser creciente, decreciente o constante.
- Puede aumentar o disminuir de diferentes maneras a lo largo del tiempo (linealmente, exponencialmente, etc.).

In [4]:
import numpy as np
import pandas as pd
import hvplot.pandas

time = np.arange(144)
trend = time * 2.65 +100

timeseries = pd.DataFrame({'Time': time, 'Trend': trend})

timeseries.hvplot.line(
    x='Time', y='Trend', grid=True, color='red', line_width=1,
    xlabel='Tiempo', ylabel='Pasajeros'
).opts(active_tools=[])

:Curve   [Time]   (Trend)

### Estacionalidad (_seasonality_)

- __Fluctuaciones periódicas__ en datos de series temporales que ocurren a __intervalos regulares__ debido a factores estacionales.
- Se caracteriza por patrones consistentes y predecibles durante un período específico (e.g., diario, mensual, trimestral, anual).

Puede estar impulsada por muchos factores.
- Eventos que ocurren de forma natural, como fluctuaciones climáticas causadas por la época del año.
- Procedimientos comerciales o administrativos, como el inicio y el final de un año escolar.
- Comportamiento social o cultural, por ejemplo, días festivos o celebraciones religiosas.

In [5]:
seasonal = 20 + np.sin( time * 0.5) * 20

timeseries['Seasonal'] = seasonal

timeseries.hvplot.line(
    x='Time', y='Seasonal', grid=True, color='orange', line_width=1,
    xlabel='Tiempo', ylabel='Pasajeros'
).opts(active_tools=[])

:Curve   [Time]   (Seasonal)

### Residuos (_residuals_)

- Los residuos son las __fluctuaciones aleatorias__ que quedan después de remover la tendencia y la estacionalidad de la serie temporal original.
- No debería observarse una tendencia o un patrón estacional en los residuos.
- Representan fluctuaciones a corto plazo, bastante impredecibles.

In [6]:
residuals = np.random.normal(loc=0.0, scale=3, size=len(time))

timeseries['Residuals'] = residuals

timeseries.hvplot.line(
    x='Time', y='Residuals', grid=True, color='green', line_width=1,
    xlabel='Tiempo', ylabel='Pasajeros'
).opts(active_tools=[])

:Curve   [Time]   (Residuals)

### Modelos de descomposición

Los componentes de las series temporales se pueden descomponer con los siguientes modelos:
- __Descomposición aditiva__
- Descomposición multiplicativa
- Descomposición pseudoaditiva

Los modelos aditivos suponen que la serie temporal observada es la __suma de sus componentes__:

$$X(t) = T(t) + S(t) + R(t) $$

donde,

- $X(t)$ es la serie temporal
- $T(t)$ es la tendencia
- $S(t)$ es la estacionalidad
- $R(t)$ es el residuo

In [7]:
additive = trend + seasonal + residuals

timeseries['Additive'] = additive

timeseries.hvplot.line(
    x='Time', y='Additive', grid=True, color='blue', line_width=1,
    xlabel='Tiempo', ylabel='Pasajeros'
).opts(active_tools=[])

:Curve   [Time]   (Additive)

### Descomposición aditiva

La librería [statsmodels](https://www.statsmodels.org/) proporciona una implementación del método de descomposición clásico en una función llamada `seasonal_decompose()`. Requiere que especifique si el modelo es aditivo o multiplicativo.

- Es necesario especificar un número entero que represente la estacionalidad principal de los datos.
- Al observar el componente estacional de la serie, el período tiene una duración aproximada de 12 pasos de tiempo.

In [8]:
from statsmodels.tsa.seasonal import seasonal_decompose

descomposicion = seasonal_decompose(x=additive, model='additive', period=12)

pd.DataFrame({
    'Original': additive,
    'Trend': descomposicion.trend,
    'Seasonal': descomposicion.seasonal,
    'Residuos': descomposicion.resid
}).hvplot(
    width=800, height=150, subplots=True, shared_axes=False, line_width=1, grid=True
).cols(1)

:NdLayout   [Variable]
   :Curve   [index]   (value)

## Trabajando con series temporales en Python

- ___Time stamps___ hacen referencia a momentos particulares en el tiempo (e.g., 4 de julio de 2015 a las 7:00 a.m.).
- ___Time intervals___ hace referencia a un período de tiempo entre un punto de inicio y fin en particular (e.g., año 2015). 
- ___Time periods___ generalmente se refieren a un caso especial de intervalos de tiempo en los que cada intervalo es de longitud uniforme y no se superpone (e.g., períodos de 24 horas que comprenden días).
- ___Time deltas___ hace referencia a un período de tiempo exacto (e.g., 22.56 segundos).

[Pandas](https://pandas.pydata.org) proporciona el objeto `Timestamp`, que permite construir `DatetimeIndex` para indexar datos en una Serie o DataFrame

Por ejemplo, desde cadena de texto es posible __crear un objeto__ `Timestamp`, y luego a partir de este, obtener el nombre del día de la semana: 

In [9]:
import pandas as pd

fecha = pd.to_datetime("18th of september, 1810")
fecha.day_name()

'Tuesday'

Además, es posible aplicar __operaciones vectorizadas__ (estilo NumPy) __directamente sobre el objeto__ `Timestamp`. Por ejemplo, para crear una secuencia nuevas fechas:

In [10]:
import numpy as np

fecha + pd.to_timedelta(np.arange(15), 'D')

DatetimeIndex(['1810-09-18', '1810-09-19', '1810-09-20', '1810-09-21',
               '1810-09-22', '1810-09-23', '1810-09-24', '1810-09-25',
               '1810-09-26', '1810-09-27', '1810-09-28', '1810-09-29',
               '1810-09-30', '1810-10-01', '1810-10-02'],
              dtype='datetime64[ns]', freq=None)

Donde se destaca la utilidad de las series temporales de Pandas, es para __indexar datos__. Por ejemplo, para construir Series con datos indexados por tiempo, y __filtrar datos__ a partir de sus indíces:

In [11]:
index = pd.DatetimeIndex(['2013-08-21', '2013-10-01',
                          '2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series(np.arange(6), index=index)

# filtrar para un periodo a partir de límites
data['2014-07-04':'2015-01-01']

2014-07-04    2
2014-08-04    3
dtype: int64

In [12]:
# filtrar por una jerarquía de nivel superior
data['2014']

2014-07-04    2
2014-08-04    3
dtype: int64

### Secuencias regulares y frecuencias

Para que la creación de secuencias de fechas regulares, Pandas ofrece diferentes funciones para crear secuencias de ___period___, de ___timestamp___ y de ___delta___ 
- `pd.date_range()` para marcas de tiempo, 
- `pd.period_range()` para períodos 
- `pd.timedelta_range()` para deltas de tiempo.

`pd.date_range()` requiere de una fecha inicio, una fecha de finalización y una frecuencia (por defecto es día `"D"`:

In [13]:
# inicio y fin
pd.date_range('2015-07-03', '2015-07-10')
# inicio y cantidad de periodos
pd.date_range('2015-07-03', periods=8)
# inicio, cantidad de periodos y frecuencia (horas)
pd.date_range('2015-07-03', periods=8, freq='H')

DatetimeIndex(['2015-07-03 00:00:00', '2015-07-03 01:00:00',
               '2015-07-03 02:00:00', '2015-07-03 03:00:00',
               '2015-07-03 04:00:00', '2015-07-03 05:00:00',
               '2015-07-03 06:00:00', '2015-07-03 07:00:00'],
              dtype='datetime64[ns]', freq='H')

Para crear secuencias regulares de valores de ___Period___ o ___Timedelta___, son útiles las funciones ``pd.period_range()`` y ``pd.timedelta_range()``,

In [14]:
pd.period_range('2015-07', periods=8, freq='M')

PeriodIndex(['2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
             '2016-01', '2016-02'],
            dtype='period[M]')

In [15]:
pd.timedelta_range(0, periods=10, freq='H')

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00'],
               dtype='timedelta64[ns]', freq='H')

A partir de códigos es posible especificar cualquier espaciado de frecuencia deseado ([mas información](https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects)).

| Code   | Description         | Code   | Description          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |        |                      |
| ``S``  | Seconds             |        |                      |
| ``L``  | Milliseonds         |        |                      |
| ``U``  | Microseconds        |        |                      |
| ``N``  | nanoseconds         |        |                      |

Los códigos se pueden combinar con números para especificar otras frecuencias. Por ejemplo, para una frecuencia de 2 horas y 30 minutos,

In [16]:
pd.timedelta_range(0, periods=9, freq="2H30T")

TimedeltaIndex(['0 days 00:00:00', '0 days 02:30:00', '0 days 05:00:00',
                '0 days 07:30:00', '0 days 10:00:00', '0 days 12:30:00',
                '0 days 15:00:00', '0 days 17:30:00', '0 days 20:00:00'],
               dtype='timedelta64[ns]', freq='150T')

## Ejemplo de análisis de series temporales

En esta sección, se utilizan [Open Power System Data (OPSD)](https://open-power-system-data.org), iniciativa que recopila y publica datos sobre los sistemas de energía de Europa occidental para ayudar al desarrollo de modelos de sistemas de energía.

Para comprender un análisis elemental de series temporales. 
- Se analiza las estructuras de datos de series temporales, 
- Se análiza la indexación basada en el tiempo y algunas formas de visualización de series temporales.

In [17]:
import pandas as pd

opsdata = pd.read_csv('examples/opsd_germany_daily.csv', parse_dates=['Date'])
opsdata.set_index('Date', inplace=True)
opsdata.head(5)

,Consumption,Wind,Solar,Wind+Solar
Date,,,,
2006-01-01,1069.184,NaN,NaN,NaN
2006-01-02,1380.521,NaN,NaN,NaN
2006-01-03,1442.533,NaN,NaN,NaN
2006-01-04,1457.217,NaN,NaN,NaN
2006-01-05,1477.131,NaN,NaN,NaN


- __Date__: Contiene la fecha con formato `aaaa-mm-dd`
- __Consumption__: Indica el consumo diario de electricidad en GWh. 
- __Wind__: Indica la producción de energía eólica en GWh.
- __Solar__: Indica la producción de energía solar en GWh.
- __Wind+Solar__: Representa la suma de la producción de energía solar y eólica en GWh.

A partir de la serie temporal (index) es posible agregar nuevas columnas: año, mes y el nombre del día de la semana.

In [18]:
opsdata['year'] = opsdata.index.year
opsdata['Month'] = opsdata.index.month
opsdata['Weekday Name'] = opsdata.index.day_name()
opsdata

,Consumption,Wind,Solar,Wind+Solar,year,Month,Weekday Name
Date,,,,,,,
2006-01-01,1069.18400,NaN,NaN,NaN,2006,1,Sunday
2006-01-02,1380.52100,NaN,NaN,NaN,2006,1,Monday
2006-01-03,1442.53300,NaN,NaN,NaN,2006,1,Tuesday
2006-01-04,1457.21700,NaN,NaN,NaN,2006,1,Wednesday
2006-01-05,1477.13100,NaN,NaN,NaN,2006,1,Thursday
...,...,...,...,...,...,...,...
2017-12-27,1263.94091,394.507,16.530,411.037,2017,12,Wednesday
2017-12-28,1299.86398,506.424,14.162,520.586,2017,12,Thursday
2017-12-29,1295.08753,584.277,29.854,614.131,2017,12,Friday


In [19]:
import hvplot.pandas

opsdata.hvplot.scatter(
    y=['Consumption', 'Wind', 'Solar'],
    value_label="Consumo diario (GWh)",
    height=400,
    width=900,
    size=2,
    legend='bottom_left',
    grid=True
).opts(
    active_tools=[]
)

:NdOverlay   [Variable]
   :Scatter   [Date]   (Consumo diario (GWh))

Se observa los siguientes patrones:
1. El consumo de electricidad se puede dividir en dos patrones distintos:
    - Un grupo de aproximadamente 1.400 GWh y más.
    - Otro grupo de aproximadamente menos de 1.400 GWh.

2. La producción solar es mayor en verano (junio - agosto) y menor en invierno (diciembre - febrero). 
3. A lo largo de los años, parece haber habido una fuerte tendencia al alza en la producción de energía eólica.

Análizando en detalle un el año 2016:

In [20]:
opsdata.loc['2016', 'Consumption'].hvplot.line(
    line_width=1,
    ylabel='Consumo diario (GWh)',
    grid=True
).opts(
    active_tools=[]
)

:Curve   [Date]   (Consumption)

El gráfico muestra una disminución drástica del consumo de electricidad a finales de año (diciembre) y durante agosto. 
 
A continuación, se examina el mes de diciembre de 2016:

In [21]:
opsdata_dec = opsdata.loc['2016-12', 'Consumption'] 
line = opsdata_dec.hvplot.line(
    ylabel='Consumo diario (GWh)',
    grid=True,
    line_width=1)

scatter = opsdata_dec.hvplot.scatter(
    size=10,
    legend='top_right'
)

(line * scatter).opts(active_tools=[])

:Overlay
   .Curve.Consumption   :Curve   [Date]   (Consumption)
   .Scatter.Consumption :Scatter   [Date]   (Consumption)

Se observa que, 
- El consumo de electricidad es mayor durante los días laborables y menor durante los fines de semana. 
- El consumo de electricidad fue más bajo el día de Navidad, probablemente porque la gente estaba ocupada celebrando. Después de Navidad, el consumo aumentó.

Agrupando series temporales

In [22]:
w, h = 400, 400
boxs = opsdata.hvplot.box(
    y='Consumption', by='Month', 
    width=w, height=h, grid=True).opts(active_tools=[])
for var in ['Wind','Solar']:
    box = opsdata.hvplot.box(
        y=var, by='Month', width=w, height=h, grid=True)
    boxs = boxs + box.opts(active_tools=[])

boxs

:Layout
   .BoxWhisker.I   :BoxWhisker   [Month]   (Consumption)
   .BoxWhisker.II  :BoxWhisker   [Month]   (Wind)
   .BoxWhisker.III :BoxWhisker   [Month]   (Solar)

El gráfico muestra que,
 
- El consumo de electricidad es generalmente mayor en invierno (dic. - feb.) y menor en verano (jun. - ago.). 
- La producción solar es mayor durante el verano (jun. - ago.). 
- Existen valores atípicos asociados con el consumo de electricidad, la producción eólica y la producción solar.

El consumo energético agrupado por día de la semana:

In [23]:
opsdata.hvplot.box(
    y='Consumption',
    by='Weekday Name',
    grid=True
).opts(active_tools=[])

:BoxWhisker   [Weekday Name]   (Consumption)

El gráfico muestra que el consumo de electricidad es mayor entre semana que los fines de semana. Curiosamente, hay más valores atípicos entre semana.

En general, se hace necesario realizar un remuestreo (_resampling_) de los datos con frecuencias más bajas o más altas. Este remuestreo se realiza en función de operaciones de agregación o agrupación. 

Por ejemplo, para realizar un remuestreo de la serie en función de la media semanal:

In [24]:
media_semanal = opsdata[['Consumption', 'Wind', 'Solar']].resample('W').mean()
media_semanal.head(5)

,Consumption,Wind,Solar
Date,,,
2006-01-01,1069.184000,NaN,NaN
2006-01-08,1381.300143,NaN,NaN
2006-01-15,1486.730286,NaN,NaN
2006-01-22,1490.031143,NaN,NaN
2006-01-29,1514.176857,NaN,NaN


In [25]:
start, end = '2016-01', '2016-06'

diaria_line = opsdata.loc[start:end, 'Solar'].hvplot.line(line_width=1, color='lightblue', label='Diaria', grid=True)
diaria_scat = opsdata.loc[start:end, 'Solar'].hvplot.scatter(line_width=1, size=5,color='blue', label='Diaria')

semanal_line = media_semanal.loc[start:end, 'Solar'].hvplot.line(line_width=1, color='orange', label='Media semanal')
semanal_scat = media_semanal.loc[start:end, 'Solar'].hvplot.scatter(line_width=1, size=5, color='red', label='Media semanal')

(diaria_line * diaria_scat * semanal_line * semanal_scat).opts(
    ylabel='Producción de energía solar (GWh)', active_tools=[], legend_position='top_left'
)

:Overlay
   .Curve.Diaria          :Curve   [Date]   (Solar)
   .Scatter.Diaria        :Scatter   [Date]   (Solar)
   .Curve.Media_semanal   :Curve   [Date]   (Solar)
   .Scatter.Media_semanal :Scatter   [Date]   (Solar)

La serie temporal promedio semanal aumenta con el tiempo y es mucho más suave que la serie temporal diaria.

## Referencias

1. J. VanderPlas, Python Data Science Handbook. in Essential Tools for Working with Data. O’Reilly Media, Inc., 2017.
2. T. C. Mills, Applied Time Series Analysis. Academic Press, 2019.
  
  
  
  